In [289]:
import pandas as pd
import numpy as np
import os
import datetime
from datetime import date
from datetime import datetime
from sklearn.model_selection import train_test_split

In [290]:
# setting saving directory
today = date.today()
now = datetime.now()
print (today)
print (now)
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S") #do not put : or similar 
print("date and time =", dt_string)	    
root = "C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs"

#saving function
def saveAsCSV(df,name,rootdir = root):
    """
    df = dataframe
    name = name for saving in format: 'thisisthename.csv'
    rootdir = local saving directory, default is root
    """
    savepath = rootdir +'/' + dt_string+'_'+ name 
    df.to_csv ( savepath, index = False, header=True)

#loading function
def loadCSV(name,rootdir = root):
    """
    rootdir = local saving directory, default is root
    name = exact name (including datetime and extention .csv)
    """
    loadpath = rootdir +'/'+ name 
    print ("loading: ",loadpath)
    df = pd.read_csv(loadpath)
    return df
    

2020-06-20
2020-06-20 14:19:49.610042
date and time = 20-06-2020_14-19-49


In [310]:
#here all the functions (for when we will use many df)

def keep1(arrayorint):
    """
    array = array of numbers
    """
    if (type(arrayorint) != int):
        new_list =[] 
        for num in arrayorint:
            first_digit = int(str(num)[:1]) 
            new_list.append(first_digit)
        new = np.array(new_list)
    else: 
        new = int(str(arrayorint)[:1]) 
    return new

def keepOne(arrayorint):
    """
    array = array of numbers
    """
    if (type(arrayorint) != int):
        new_list =[] 
        for num in arrayorint:
            first_digit = int(np.floor(num / 1000))
            new_list.append(first_digit)
        new = np.array(new_list)
    else: 
        new = int(np.floor(arrayorint / 1000))
    return new


def infoDataFrame(dt_test):
    #how many entries in this dataset? 
    tot_rows= len(dt_test)
    print ("tot num entries in dataset: ",tot_rows)

    #how many hh in this dataset? 
    tot_hh  = len(dt_test.serial.unique())
    print ("tot num hh in dataset: ",tot_hh)

    #how many people in this dataset? 
    tot_ppl = len(dt_test.person_id.unique())
    print ("tot num people in dataset: ",tot_ppl )

    #how many rows in training and how many in testing? 
    train = np.floor(tot_rows*0.8)
    print ("train: %i,   test: %i" %(train,tot_rows-train))

def addUniquePersonID(dt_test):
    dt_test["person_id"] = dt_test["serial"] + dt_test["pnum"]/100
    return dt_test


def addNextActivity(df_test):
    whatisnext = []
    for i in range(len(df_test)):

        # Get details of row
        current_person_id = df_test['person_id'][i]
        current_activity = df_test['whatdoing_exact'][i]

        if i < (len(df_test)-1):
            #check next row
            next_person_id = df_test['person_id'][i+1]
            next_activity = df_test['whatdoing_exact'][i+1]

            # check and add
            if (current_person_id == next_person_id):
                whatisnext.append(next_activity)

            else: #assumption: we assume the activity is continued
                whatisnext.append(current_activity)

        else: #assumption: we assume the activity is continued
            #print (i)
            whatisnext.append(current_activity)

    whatisnext = pd.Series(whatisnext, name = 'Next_activity')
    new_column_df = whatisnext.to_frame()    

    #we check for missing values, if the new column is shorter--> erron in the concatenation!
    if (len(df_test) != len(whatisnext)):
        print("original len: %i, modified len: %i"  %(len(df_test) ,len(whatisnext)))

    #concat on axis = 1 --> we add one column
    df_test = pd.concat([df_test, new_column_df],axis=1)
    return df_test

def createTransitionMatrix (df_test):
    #get the indexes 
    all_indexes = df_test.index.values.tolist()

    #sizing 
    all_next_act= df_test['Next_activity']
    simplified_all_next_act = keepOne(all_next_act)
    unique_next_act =np.unique(simplified_all_next_act)
    #print (unique_next_act)
    n = len(unique_next_act)
    print ("Number of unique Activities:",n)

    # Init
    total_tp = 144
    P_t = np.zeros([total_tp, n, n]) # <- 3D array for 144 * transition matrices

    # For debugging
    test = False

    # Loop through all activity entries again
    for i in all_indexes:
        #here we get double codes
        t = df_test['time_period'][i] # <- % 144 to take all days in the years
        j = df_test['whatdoing_exact'][i] # <- current state
        k = df_test['Next_activity'][i]  # <- next state
        #print (t,j,k)
        
        #here we simplify them into one (eg "31" ->"3", "32"->"3", "99"->9)
        j_simply = keepOne(j.item())
        k_simply = keepOne(k.item())
        #print (type(t),type(j_simply),type(k_simply))

        #we take their positional indexes from unique_next_act
        j_index= np.where(unique_next_act == j_simply) 
        k_index= np.where(unique_next_act == k_simply) 

        #to increase the right bin
        P_t[t, j_index, k_index] += 1 # <- add one to probability, we'll divide later
        
        #if test:
            #print(activ_array[i, :])
            #print("At time %0.0f, the person is transitioning from %0.0f to %0.0f" % (t, j, k))
            #print("------")
            
    for t in range(total_tp):
        P_t[t] = (P_t[t].T/np.sum(P_t[t], axis=1)).T # <- transpose and transpose back to match axes with division operator
        for row in range(n):
            if np.isnan(P_t[t, row]).any(): # <- replace nans with 0s
                P_t[t, row] = np.zeros(n)
       
    return P_t
    

In [293]:
# Load data
df_adult1 = loadCSV('20-06-2020_14-06-26_df_adult1.csv')

loading:  C:/Users/arita/Documents/private/Data Engineering/2semestre/Stochastic_modeling/Project/data/outputs/20-06-2020_14-06-26_df_adult1.csv


In [294]:
#peek
df_adult1.head(20)

,old_time_index,serial,pnum,Date,Time,t_start,eptime,whatdoing_exact,sex,age,New_Time,time_period
0,502,11011220,3,2015-01-18,18:40:00,2015-01-18 18:40:00,10.0,5110,2,23,18:40:00,112
1,503,11011220,3,2015-01-18,18:50:00,2015-01-18 18:50:00,10.0,5140,2,23,18:50:00,113
2,503,11011220,3,2015-01-18,18:50:00,2015-01-18 18:50:00,10.0,5140,2,23,19:00:00,114
3,503,11011220,3,2015-01-18,18:50:00,2015-01-18 18:50:00,10.0,5140,2,23,19:10:00,115
4,503,11011220,3,2015-01-18,18:50:00,2015-01-18 18:50:00,10.0,5140,2,23,19:20:00,116
5,503,11011220,3,2015-01-18,18:50:00,2015-01-18 18:50:00,10.0,5140,2,23,19:30:00,117
6,504,11011220,3,2015-01-18,19:40:00,2015-01-18 19:40:00,10.0,3240,2,23,19:40:00,118
7,504,11011220,3,2015-01-18,19:40:00,2015-01-18 19:40:00,10.0,3240,2,23,19:50:00,119
8,505,11011220,3,2015-01-18,20:00:00,2015-01-18 20:00:00,10.0,8120,2,23,20:00:00,120
9,506,11011220,3,2015-01-18,20:10:00,2015-01-18 20:10:00,10.0,8210,2,23,20:10:00,121


In [295]:
# Add unique person identifier:
#df_adult1["person_id"] = df_adult1["serial"] + df_adult1["pnum"]/100
df_adult1 = addUniquePersonID(df_adult1 )

In [298]:
# sort columns, drop ones that are not useful
cols = list(df_adult1)
cols
newcols = ['old_time_index',
 'serial',
 'pnum',
 'person_id',
 'Date',
 'New_Time',
 'time_period',
 'whatdoing_exact',
 'sex',
 'age',
 'Time',
 't_start',
 'eptime']
 #'time_period']
df_adult1 = df_adult1.ix[:,newcols] #says deprecated, but works!

C:\Users\arita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [299]:
# drop columns that are not useful
df_adult1 = df_adult1.drop(columns=['Time','eptime'])
df_adult1.head()

,old_time_index,serial,pnum,person_id,Date,New_Time,time_period,whatdoing_exact,sex,age,t_start
0,502,11011220,3,11011220.03,2015-01-18,18:40:00,112,5110,2,23,2015-01-18 18:40:00
1,503,11011220,3,11011220.03,2015-01-18,18:50:00,113,5140,2,23,2015-01-18 18:50:00
2,503,11011220,3,11011220.03,2015-01-18,19:00:00,114,5140,2,23,2015-01-18 18:50:00
3,503,11011220,3,11011220.03,2015-01-18,19:10:00,115,5140,2,23,2015-01-18 18:50:00
4,503,11011220,3,11011220.03,2015-01-18,19:20:00,116,5140,2,23,2015-01-18 18:50:00


In [300]:
infoDataFrame(df_adult1)

tot num entries in dataset:  97994
tot num hh in dataset:  383
tot num people in dataset:  518
train: 78395,   test: 19599


In [301]:
#BEFORE splitting the dataset we should create a column with NEXTwhatisdoing 
#for each household, for each person
#giving that the lines are ordered in theory checking the pnum would be enough, but anyway we have the person_id!
df_adult1 = addNextActivity(df_adult1)

In [302]:
#check
df_adult1
saveAsCSV(df_adult1,'df_adult1_withnextactivity')

In [303]:
#from sklearn.model_selection import train_test_split
# split dataset df_adult1 in train and test
train_df_adult1, test_df_adult1 = train_test_split(df_adult1, test_size=0.2)

saveAsCSV(train_df_adult1,'train_df_adult1')
saveAsCSV(test_df_adult1,'test_df_adult1')

In [304]:
len(train_df_adult1)
train_df_adult1.head(25)
#perfect!

,old_time_index,serial,pnum,person_id,Date,New_Time,time_period,whatdoing_exact,sex,age,t_start,Next_activity
81428,138058,23080615,3,23080615.03,2015-07-12,17:50:00,107,8210,2,20,2015-07-12 17:40:00,8210
71634,123524,20051013,2,20051013.02,2015-10-25,03:30:00,21,110,1,28,2015-10-25 03:20:00,110
94892,159525,45051018,2,45051018.02,2015-10-25,04:40:00,28,110,2,21,2015-10-25 03:00:00,110
10219,23836,12190114,1,12190114.01,2015-02-21,19:40:00,118,210,1,23,2015-02-21 19:00:00,210
33560,59156,15190116,3,15190116.03,2015-02-24,14:00:00,84,7231,2,27,2015-02-24 14:00:00,7231
85402,145068,30051006,2,30051006.02,2015-10-26,23:10:00,139,110,2,23,2015-10-26 21:50:00,110
11430,25278,12230217,3,12230217.03,2015-03-10,16:50:00,101,7300,1,21,2015-03-10 15:00:00,7300
75232,128725,20160312,1,20160312.01,2015-03-20,19:10:00,115,8210,1,25,2015-03-20 19:10:00,8210
7413,17570,12081205,2,12081205.02,2015-01-10,14:40:00,88,2120,2,21,2015-01-10 14:40:00,2120
12814,27019,12260119,1,12260119.01,2015-03-14,07:30:00,45,110,2,28,2015-03-14 03:00:00,110


In [307]:
all_indexes = df_test.index.values.tolist()
#print (np.unique (all_indexes))
train_df_adult1['time_period'][0]

112

In [311]:
P_t_adult1 = createTransitionMatrix(train_df_adult1)

Number of unique Activities: 10


C:\Users\arita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:134: RuntimeWarning: invalid value encountered in true_divide


In [312]:
# looks good!
print ((P_t_adult1))

[[[1.         0.         0.         ... 0.         0.         0.        ]
  [0.         1.         0.         ... 0.         0.         0.        ]
  [0.         0.         1.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 1.         0.         0.        ]
  [0.14285714 0.         0.         ... 0.         0.85714286 0.        ]
  [0.         0.         0.         ... 0.         0.         1.        ]]

 [[0.9969697  0.         0.         ... 0.         0.00151515 0.00151515]
  [0.         1.         0.         ... 0.         0.         0.        ]
  [0.         0.         1.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 1.         0.         0.        ]
  [0.         0.         0.         ... 0.         1.         0.        ]
  [0.         0.         0.         ... 0.         0.         1.        ]]

 [[0.99552239 0.         0.         ... 0.00298507 0.         0.        ]
  [0.         1.      

In [313]:
import os
os.getcwd()

'C:\\Users\\arita\\Documents\\GitHub\\02443-SimSolarEnergy\\code'

In [315]:
import pickle
#lets pickle the np array
with open( dt_string+'_'+'P_t_adult1.pkl','wb') as f:
    pickle.dump(P_t_adult1, f)
    

In [320]:
all_act = df_adult1.whatdoing_exact
simplified_all_act = keepOne(all_act)
unique_act =np.unique(simplified_all_act)

In [321]:
unique_act

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])